In [1]:
from __future__ import print_function
from code import *

In [2]:
raw_train = read_file("adult.data.txt")
raw_test = read_file("adult.test.txt")

In [3]:
average_positive_target_rate = get_average_positive_target_rate(raw_train)
X_train, Y_train = make_dataset(raw_train, average_positive_target_rate)
X_train, Y_train = unique_dataset(X_train, Y_train)

In [4]:
X_train.shape, Y_train.shape

((15210, 75), (15210,))

In [5]:
train_size = 500
test_size = 100
permupation = np.random.permutation(len(X_train))
train_indices = permupation[:train_size]
test_indices = permupation[train_size:(train_size + test_size)]

train_x = X_train[train_indices]
train_y = Y_train[train_indices]
test_x = X_train[test_indices]
test_y = Y_train[test_indices]

In [6]:
def make_kfold(X, Y, folds):
    kf = KFold(n_splits=folds)
    metrics = []
    stats = []
    for train_index, test_index in kf.split(X):
        train_x, test_x = X[train_index], X[test_index]
        train_y, test_y = Y[train_index], Y[test_index]
        
        pos_stats, neg_stats = lattices_stats(train_x, train_y, test_x)

        stats.append((pos_stats, neg_stats))
        
        predictions = [np.zeros_like(test_y)]
        predictions.append([
            1 if pos_stat[0] > neg_stat[0] else 0
            for pos_stat, neg_stat in zip(pos_stats, neg_stats)
        ])
        predictions.append([
            1 if pos_stat[1] > neg_stat[1] else 0
            for pos_stat, neg_stat in zip(pos_stats, neg_stats)
        ])
        predictions.append([
            1 if pos_stat[2] > neg_stat[2] else 0
            for pos_stat, neg_stat in zip(pos_stats, neg_stats)
        ])
        predictions.append([
            1 if pos_stat[3] > neg_stat[3] else 0
            for pos_stat, neg_stat in zip(pos_stats, neg_stats)
        ])
        predictions.append([
            1 if pos_stat.sum() > neg_stat.sum() else 0
            for pos_stat, neg_stat in zip(pos_stats, neg_stats)
        ])
        predictions.append([
            1 if (pos_stat * np.array([1, 33, 100, 50])).sum() > (neg_stat * np.array([1, 33, 100, 50])).sum() else 0
            for pos_stat, neg_stat in zip(pos_stats, neg_stats)
        ])
                
        metrics.append([
            accuracy_score(prediction, test_y)
            for prediction in predictions
        ] + [get_catboost_acc(train_x, train_y, test_x, test_y)])
    return np.array(metrics), stats

In [ ]:
#metrics, stats = make_kfold(train_x, train_y, 5)
metrics, stats = make_kfold(X_train, Y_train, 5)

0


In [ ]:
for metric in metrics:
    print("{} {} {} {} {} {} {} {}".format(*metric))

In [ ]:
print("{} {} {} {} {} {} {} {}".format(*metrics.mean(axis=0)))